In [ ]:
from glob import glob
import os
from pathlib import Path
from PipeLine import *

In [ ]:
from dask.distributed import Client
from dask import delayed
from dask import compute
client = Client(n_workers=4)
client

In [ ]:
# Path to bug extraction folders/files
database = '/Users/amirhsi_mini/extractions/'
#database = './extractions/'
bug_groups = glob(database+"/N*-extraction/")
geometry = 'cylindrical'

In [ ]:
print(bug_groups)

In [ ]:
def whole_group_analyze_bug(extraction_db, extraction_db_name, analysis_db_name, simulation_type, geometry):
    """
    whole_group_analyze_bug create ensemble/ensemble-averaged files and folders for bug files.
    """
    sim_csvs = glob(extraction_db+"/N*/N*.csv")
    analysis_db = extraction_db.split(extraction_db_name)[0]+analysis_db_name+"/"
    analysis_name = extraction_db.split("/")[-2].split('-')[0]+'-'+simulation_type+'-analysis'
    analysis_path = analysis_db+analysis_name
    Path(analysis_path).mkdir(parents=True, exist_ok=False)
    analysis_save_to = analysis_path+"/"
    
    analysis_ens_evg_name = extraction_db.split("/")[-2].split('-')[0]+'-'+simulation_type+'-analysis-ens_avg'
    analysis_ens_evg_path = analysis_db+analysis_ens_evg_name
    Path(analysis_ens_evg_path).mkdir(parents=True, exist_ok=False)
    analysis_ens_avg_save_to = analysis_ens_evg_path+"/"
    PipeLine.analysis_bug_sumrule(sim_csvs,geometry, analysis_save_to, analysis_ens_avg_save_to)

In [ ]:
# analyze all the bug groups at once:
simulation_type = 'bug' # bug or all
extraction_db_name = "extractions"
analysis_db_name = "analysis_bug"
geometry = 'cylindrical'
analysis_delayed = []
for bug_group in bug_groups:
    analysis = delayed(whole_group_analyze_bug)(bug_group, extraction_db_name, analysis_db_name, simulation_type, geometry)
    analysis_delayed.append(analysis)

In [ ]:
%%time
results = compute(analysis_delayed)

In [ ]:
database = '/Users/amirhsi_mini/logs/'
log_groups = glob(database+"/N*-logs/")
geometry = 'cylindrical'

In [ ]:
# analyze log files
for log_group in log_groups:
    log_files = glob(log_group+'/N*.log')
    log_files = PipeLine.file_reader(log_files,extensions=['.log'])
    sample_log = log_files[0][0].split("/")[-1]
    details_out , runtime_out = PipeLine.log_outputs(sample_log, geometry)
    PipeLine.lammps_log_details(log_files, details_out , runtime_out)

In [ ]:
# create one dataframe of all the properties files:
database_path = '/Users/amirhsi_mini/trjs_analysis_bug/'
properties_files = glob(database_path+"N*-bug-analysis/N*-properties.csv")
properties_files = PipeLine.file_reader(properties_files,extensions=['-properties.csv'])
properties_all_in_one = PipeLine.properties_all_in_one(properties_files, save_to=database_path, round_to=4, index_col=0)
properties_files_ens_avg = glob(database_path+"N*-bug-analysis-ens_avg/N*-properties-ens_avg.csv")
properties_files_ens_avg = PipeLine.file_reader(properties_files_ens_avg,extensions=['-properties-ens_avg.csv'])
properties_all_in_one_ens_avg = PipeLine.properties_all_in_one(properties_files_ens_avg, ens_avg=True, norm_func=PipeLine.cylindrical_norm, save_to=database_path, round_to=4, index_col=0)